In [15]:
%load_ext Cython
import timeit

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


# 2: Cython language basics

**Variable types**

In [2]:
%%cython

def cy_summation(n):
    a = 0
    for i in range(n):
        a += i
    return a

In [3]:
def py_summation(n):
    a = 0
    for i in range(n):
        a += i
    return a

In [5]:
%timeit cy_summation(10000)

396 µs ± 11.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
%timeit py_summation(10000)

557 µs ± 18.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Let's add some types

In [8]:
%%cython

def cy_summation(n):
    a = 0
    for i in range(n):
        a += i
    return a

In [9]:
%timeit cy_summation(10000)

430 µs ± 14.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


For illustration:

In [4]:
import timeit

cy_perf = timeit.timeit('cy_summation(10000)', setup='from summation import cy_summation', number=100)
py_perf = timeit.timeit('py_summation(10000)', setup='from summation import py_summation', number=100)

print('Python performance is %.6fx Cython performance' % (py_perf / cy_perf))

Python performance is 11024.926019x Cython performance


**Functions**

**cdef** can also be used to declare a C function, with the limitation that the function can't be invoked directly from Python anymore.

In [28]:
%%cython

cdef double my_function(int a, int b, double f):
    return (a + b) / f

print(my_function(10, 20, 3.00))

10.0


In [9]:
print(my_function(10, 20, 3.00))

NameError: name 'my_function' is not defined

**cpdef**, however, wraps the 'pure' c function in a Python object, thereby avoiding the limitations imposed by cdef. In simple terms, cpdef invokes the pure C function from a Python function for you.

In [31]:
%%cython

cpdef double my_wrapped_function(int a, int b, double f):
    return (a + b) / f

print(my_wrapped_function(10, 20, 3.00))

10.0


In [32]:
print(my_wrapped_function(10, 20, 3.00))

10.0


**cpdef** does come with a bit of extra overhead.

**Complex datatypes**

In [27]:
%%cython

ctypedef long long PlacementId

cdef enum AppOrSite:
    app = 0
    site = 1

ctypedef struct Bid:
    signed int campaign_id
    signed int tactic_id
    signed int banner_id
    float bid_price_cpm
    int ast
    PlacementId placement_id

bid  = Bid(campaign_id=10001, tactic_id=50001, banner_id=30001, bid_price_cpm=1.00, ast=AppOrSite.app, placement_id=9223372036854775807)
print(bid)

{'campaign_id': 10001, 'tactic_id': 50001, 'banner_id': 30001, 'bid_price_cpm': 1.0, 'ast': 0, 'placement_id': 9223372036854775807}


**Exceptions**

As exceptions do not exist in C, C functions do not have a way of reporting exceptions raised inside of them if they don't return Python objects.

In [7]:
%%cython

cdef int _no_negatives_please(int i):
    if i < 0:
        raise Exception("no negatives!")
    else:
        print("thank you")
        return -1

cpdef int no_negatives_please(int i):
    return _no_negatives_please(i)

In [8]:
print(no_negatives_please(-1))
print('still executing this stuff...')

Exception: no negatives!

0
still executing this stuff...


Exception ignored in: '_cython_magic_6a2b215c88f2ee93ec9f576bd1f6039e._no_negatives_please'
Exception: no negatives!


We can cure this with the 'except' clause which serves to propagate errors raised in Cython functions or C functions that involve C / Python API interactions.

**Directly including functions from C libraries**

In [30]:
%%cython

cdef extern from "math.h":
    double sqrt(double arg)

print(sqrt(3600))

60.0


# 3: Analysing Cython

**Annotating and profiling the Basel problem**

In [31]:
def recip_square(x):
    return 1. / x**2

def approximate_pi(n):
    result = 0.
    for i in range(1, n + 1):
        result += recip_square(i)
    return (6 * result) ** .5

%prun approximate_pi(10000000)

In [34]:
%%cython
#cython: profile=True

import cython

def cy_recip_square(x):
    return 1. / x**2

def cy_approximate_pi(n):
    result = 0.
    for i in range(1, n + 1):
        result += cy_recip_square(i)
    return (6 * result) ** .5

In [35]:
%prun -s time cy_approximate_pi(10000000)

**Legend for cProfile**

**ncalls**   
the number of calls.   
**tottime**   
the total time spent in the given function (and excluding time made in calls to sub-functions)   
**percall**   
is the quotient of tottime divided by ncalls   
**cumtime**   
is the cumulative time spent in this and all subfunctions (from invocation till exit).    
**percall**   
is the quotient of cumtime divided by primitive calls   
**filename:lineno(function)**   
provides the respective data of each function

# 4: Breaking Cython

In [28]:
%%cython

cpdef int incr(int i):
    i = i + 1
    return i

In [17]:
incr(100)

101

 An int is represented by 4 bytes. The interpreter will try to fit the argument passed to incr within this range.  What happens when we invoke incr with an integer beyond this range?

In [ ]:
# Break the incr function

Python functions (functions that are not defined with cdef or cpdef) need to convert the types passed to them. The Cython compiler does this for you but only for structs, numeric types and string types. Passing other types, such as functions, will result in a compile-time error.

In [14]:
%%cython

def _execute_callback(callback):
    cdef char* s = "hello world";
    callback(s)
    return 0

cdef callback(msg):
    print('hello world')

_execute_callback(callback)

**Optimisation exercise**

In [24]:
def part_one(a, b):
    factor_a = 16807
    factor_b = 48271
    mod = 2147483647
    match_count = 0

    for i in range(40000000):
        a = a * factor_a % mod
        b = b * factor_b % mod
        binary_a = bin(a)[2:][-16:]
        binary_b = bin(b)[2:][-16:]
        if binary_a == binary_b:
            match_count += 1
        i += 1

    return match_count

In [26]:
# Timing the performance of part 1
a, b = map(lambda x: int(x[-3:]), open('aoc/inp15.txt').read().split('\n'))
%timeit -n1 print('Part 1: {}'.format(part_one(a, b)))

Part 1: 573
Part 1: 573
Part 1: 573
Part 1: 573
Part 1: 573
Part 1: 573
Part 1: 573
50.4 s ± 2.62 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
def part_two(a, b):
    factor_a = 16807
    factor_b = 48271
    mod = 2147483647
    match_count = 0
    a_numbers = []
    b_numbers = []

    while len(a_numbers) < 5000000:
        a = a * factor_a % mod
        if a % 4 == 0:
            a_numbers.append(a)

    while len(b_numbers) < 5000000:
        b = b * factor_b % mod
        if b % 8 == 0:
            b_numbers.append(b)
    i = 0
    while i < 5000000:
        if bin(a_numbers[i])[2:][-16:] == bin(b_numbers[i])[2:][-16:]:
            match_count += 1
        i += 1
    return match_count

In [25]:
# Timing the performance of part 2
a, b = map(lambda x: int(x[-3:]), open('aoc/inp15.txt').read().split('\n'))
%timeit -n1 'Part 2: {}'.format(part_two(a, b))

Part 2: 294
Part 2: 294
Part 2: 294
Part 2: 294
Part 2: 294
Part 2: 294
Part 2: 294
25.3 s ± 215 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
